# Train

<https://github.com/ConnorJL/GPT2>

尝试训练 OpenWebText 的 En corpus

## 准备

In [1]:
from IPython.display import display, Image
import sys

print(sys.version)

PYTHON = sys.executable
print(PYTHON)
PIP = f'{PYTHON} -m pip'

!{PYTHON} -V
!{PIP} -V

3.7.1 | packaged by conda-forge | (default, Feb 18 2019, 01:42:00) 
[GCC 7.3.0]
/home/liuxy/miniconda3/envs/gpt2/bin/python
Python 3.7.1
pip 19.1.1 from /home/liuxy/miniconda3/envs/gpt2/lib/python3.7/site-packages/pip (python 3.7)


## 语料

在： `/home/Public/gpt2/openwebtext/1k/*.txt`

In [23]:
import os
from glob import glob, iglob

corpus_files_pattern = os.path.abspath('../../openwebtext/1k/*.txt')

!ls {corpus_files_pattern}

/home/Public/gpt2/openwebtext/1k/train_0.txt
/home/Public/gpt2/openwebtext/1k/train_100.txt
/home/Public/gpt2/openwebtext/1k/train_101.txt
/home/Public/gpt2/openwebtext/1k/train_102.txt
/home/Public/gpt2/openwebtext/1k/train_103.txt
/home/Public/gpt2/openwebtext/1k/train_104.txt
/home/Public/gpt2/openwebtext/1k/train_105.txt
/home/Public/gpt2/openwebtext/1k/train_106.txt
/home/Public/gpt2/openwebtext/1k/train_107.txt
/home/Public/gpt2/openwebtext/1k/train_108.txt
/home/Public/gpt2/openwebtext/1k/train_109.txt
/home/Public/gpt2/openwebtext/1k/train_10.txt
/home/Public/gpt2/openwebtext/1k/train_110.txt
/home/Public/gpt2/openwebtext/1k/train_111.txt
/home/Public/gpt2/openwebtext/1k/train_112.txt
/home/Public/gpt2/openwebtext/1k/train_113.txt
/home/Public/gpt2/openwebtext/1k/train_114.txt
/home/Public/gpt2/openwebtext/1k/train_115.txt
/home/Public/gpt2/openwebtext/1k/train_116.txt
/home/Public/gpt2/openwebtext/1k/train_117.txt
/home/Public/gpt2/openwebtext/1k/train_118.txt
/home/Public/gpt

## 生成 BPM 模型

使用 [google sentencepiece](https://github.com/google/sentencepiece)

In [14]:
# 使用 SPM 训练
input_str = ','.join(fn for fn in iglob(corpus_files_pattern))
!spm_train --input={input_str} --model_type=bpe --model_prefix=openwebtext --vocab_size=16000

sentencepiece_trainer.cc(49) LOG(INFO) Starts training with : 
TrainerSpec {
  input: /home/Public/gpt2/openwebtext/1k/train_190.txt
  input: /home/Public/gpt2/openwebtext/1k/train_206.txt
  input: /home/Public/gpt2/openwebtext/1k/train_221.txt
  input: /home/Public/gpt2/openwebtext/1k/train_166.txt
  input: /home/Public/gpt2/openwebtext/1k/train_75.txt
  input: /home/Public/gpt2/openwebtext/1k/train_234.txt
  input: /home/Public/gpt2/openwebtext/1k/train_24.txt
  input: /home/Public/gpt2/openwebtext/1k/train_215.txt
  input: /home/Public/gpt2/openwebtext/1k/train_126.txt
  input: /home/Public/gpt2/openwebtext/1k/train_112.txt
  input: /home/Public/gpt2/openwebtext/1k/train_150.txt
  input: /home/Public/gpt2/openwebtext/1k/train_148.txt
  input: /home/Public/gpt2/openwebtext/1k/train_192.txt
  input: /home/Public/gpt2/openwebtext/1k/train_246.txt
  input: /home/Public/gpt2/openwebtext/1k/train_76.txt
  input: /home/Public/gpt2/openwebtext/1k/train_39.txt
  input: /home/Public/gpt2/open

输出文件是:

In [24]:
!ls openwebtext.*

openwebtext.model  openwebtext.vocab


# ConnerJL GPT2 的训练

### 生成 *.tfrecords

我修改过了 `create_trfrecords.py`，用它生成！

In [28]:
!rm -rf ../logs/*.*

%cd ../datasets/openwebtext/
try:
    !{PYTHON} create_tfrecords.py \
        -i "{corpus_files_pattern}" \
        -o ../../out/openwebtext/tfrecords \
        -n train -p 5  \
        -m ../../notebooks/openwebtext.model \
        -l ../../logs/ -s 16
finally:
    %cd ../../notebooks


/home/Public/gpt2/ConnorJL-GPT2/datasets/openwebtext
Got 250 files, divided into 50 chunks.
100%|█████████████████████████████████████████| 50/50 [1:11:59<00:00, 34.83s/it]
Done! In 4319.32s, 250 / 250 good files.
/home/Public/gpt2/ConnorJL-GPT2/notebooks


修改 `sys.path`

In [29]:
import sys

ConnorJL_GPT2_PATH = os.path.realpath('..')

display(ConnorJL_GPT2_PATH)

if ConnorJL_GPT2_PATH not in sys.path:
    sys.path.append(ConnorJL_GPT2_PATH)

'/home/Public/gpt2/ConnorJL-GPT2'

### 训练的配置文件

In [6]:
import json

json_cfg_file = "OPENWEBTEXT-GPT2-117M.json"

train_conf = {
    "n_head": 12,
    "encoder_path": os.path.join(ConnorJL_GPT2_PATH, "datasets/openwebtext"),
    "n_vocab": 50257,
    "embed_dropout": 0.1,
    "lr": 0.00025,
    "warmup_steps": 2000,
    "beta1": 0.9,
    "beta2": 0.98,
    "epsilon": 1e-9,
    "opt_name": "adam",
    "weight_decay": 0.01,
    "train_batch_size": 32,
    "attn_dropout": 0.1,
    "train_steps": 10000,
    "eval_steps": 10,
    "max_steps": 500000,
    "data_path": "../../openwebtext/",
    "scale": 0.2886751345948129,
    "res_dropout": 0.1,
    "predict_batch_size": 1,
    "eval_batch_size": 32,
    "iterations": 500,
    "n_embd": 768,
    "input": "myopenwebtext",
    "model": "GPT2",
    "model_path": "MY-GPT2-117M",
    "n_ctx": 1024,
    "predict_path": "logs/predictions.txt",
    "n_layer": 12,
}

with open(json_cfg_file, "w") as fp:
    json.dump(train_conf, fp)

### 进行训练